# Python with PostgreSQL & PostGIS

<span style="color: blue;">Note: Please always run the complete Jupyter Notebook from the beginning, as object names such as 'sql' and 'gdf' are reused in the code cells.</span>

## Libraries and Settings

In [1]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/python_postgresql_postgis


## Create database connection

In [2]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

('osm_switzerland',)


## List tables in database

In [3]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    ### not a data frame!
    ### I wonder if there wasnt a better function, like 'pd.read_sql_query(query, engine)'. or similar
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

geography_columns
geometry_columns
spatial_ref_sys
planet_osm_line
planet_osm_polygon
planet_osm_point
planet_osm_roads


## Show columns and data types of selected table

In [4]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
### this way of querying could have been applied to above codeblock too. Just an alternative.
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

column_name     data_type
0             osm_id        bigint
1            z_order       integer
2           way_area          real
3                way  USER-DEFINED
4   addr:housenumber          text
..               ...           ...
68              wood          text
69         tracktype          text
70            access          text
71    addr:housename          text
72       addr:street          text

[73 rows x 2 columns]

## Query: Select buildings for which full address is available in defined zip code areas

In [5]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8001', '8002')"""

# Query the database and store the result in a GeoDataFrame

### gpd stands for geopandas (see import section) 
        """
        The method geopanda.GeoDataFrame.from_postgis(sql_query, sql_alchemy_engine) is specifically designed to read 
        spatial data from a PostGIS-enabled PostgreSQL database. 
        It automatically converts the geometry column in the database into a GeoSeries, which is a special type 
        of pandas Series that can handle geometric data. This allows for spatial operations and analyses to be 
        performed directly on the GeoDataFrame.
        Use it over the standard pandas.read_sql_query() method when working with spatial data in a PostGIS-enabled DB
        """
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


osm_id            addr:street addr:housenumber addr:city  \
0      75684702           Spiegelgasse               18    Zürich   
1     501215139        Weinbergstrasse              42c    Zürich   
2     501215142        Weinbergstrasse              41c    Zürich   
3     157954139           Löwenstrasse               67    Zürich   
4     157954140           Löwenstrasse               69    Zürich   
...         ...                    ...              ...       ...   
2350  169980503             Mythenquai                1    Zürich   
2351   96879565  General-Wille-Strasse               10    Zürich   
2352  121542121  General-Wille-Strasse               15    Zürich   
2353  121542120  General-Wille-Strasse               11    Zürich   
2354  121542125  Alfred-Escher-Strasse               32    Zürich   

     addr:postcode    building  \
0             8001         yes   
1             8001        None   
2             8001        None   
3             8001  commercial   
4             8001  commercial   
...            ...         ...   
2350          8002         yes   
2351          8002  apartments   
2352          8002  apartments   
2353          8002         yes   
2354          8002         yes   

                                                   geom  
0     POLYGON ((8.54515 47.37206, 8.54522 47.37194, ...  
1     POLYGON ((8.54415 47.38017, 8.54417 47.3801, 8...  
2     POLYGON ((8.54403 47.38013, 8.54405 47.38006, ...  
3     POLYGON ((8.53805 47.37717, 8.53808 47.37716, ...  
4     POLYGON ((8.53813 47.37725, 8.5383 47.37718, 8...  
...                                                 ...  
2350  POLYGON ((8.53554 47.36236, 8.53555 47.36223, ...  
2351  POLYGON ((8.53291 47.36359, 8.53311 47.36353, ...  
2352  POLYGON ((8.53288 47.36326, 8.53306 47.3632, 8...  
2353  POLYGON ((8.53304 47.36316, 8.53323 47.36312, ...  
2354  POLYGON ((8.53325 47.36316, 8.53327 47.3631, 8...  

[2355 rows x 7 columns]

## Show selected features on map

<span style="color: blue;">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span style="color: blue;">Example of alternative background maps (maptiles) are:</span>
- <span style="color: blue;">EsriWorldImagery</span>
- <span style="color: blue;">EsriWorldTopoMap</span>
- <span style="color: blue;">EsriWorldGrayCanvas</span>
- <span style="color: blue;">CartoDBDarkMatter</span>
- <span style="color: blue;">CartoDBPositron</span>


In [6]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
### crs stands for Coordinate Reference System
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
### controids calculates the center of each polygon
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               ### any pre-loaded background map (called 'maptiles'), see available selection in above text-block
               tiles='EsriWorldGrayCanvas')

# Map settings, this fuction basically creates a layer ontop the map
folium.GeoJson(
    gdf,
    name='geojson',
    ### weight of the lines
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

## this adds layer control to the map and enduser
folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select coffee stores in Switzerland

In [8]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee';"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


osm_id    shop                                         name  \
0    10275946038  coffee                                 Quinta Coira   
1    11920528580  coffee                               Maison du Cafe   
2     2473998249  coffee                             Pappy John & Cie   
3    11930216622  coffee                                 Charbon Café   
4     7240932670  coffee                                    Nespresso   
..           ...     ...                                          ...   
113  11993710081  coffee                                Berger Kaffee   
114   2669450520  coffee                                       Tchibo   
115   1438791023  coffee                                    Nespresso   
116   3914484865  coffee  Nurissa Shop / Nespresso Business Solutions   
117   7562485062  coffee                                    Nespresso   

                         geom  
0     POINT (9.53125 46.8485)  
1    POINT (8.98798 45.87081)  
2    POINT (6.65236 46.52688)  
3     POINT (6.6277 46.51435)  
4    POINT (6.69463 46.50053)  
..                        ...  
113  POINT (7.62597 46.87853)  
114  POINT (7.62848 46.75757)  
115  POINT (7.63094 46.75724)  
116  POINT (7.28529 47.16039)  
117  POINT (7.59167 47.55363)  

[118 rows x 4 columns]

## Show selected features on map

In [9]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
### together with the centerpoint location, zoom level at the beginning and the background map
m = folium.Map(location=[lat, lon], 
               zoom_start=9, 
               tiles='EsriWorldTopoMap')

# Map settings
### apparently if no other shape is defined, the default is a blue-white marker (see below)
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

### standard piece, adds controls to the map. I guess the zoom level (top left) and the layer control (top right) ...?
folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span style="color: blue;">Note:</span>

<span style="color: blue;">For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span style="color: blue;">In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span style="color: blue;">The WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) were derived from: https://tools.retorte.ch/map.</span>


In [10]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
### definitely revise this query! It introduces some kind of a calculated column 'ST_Distance' 
### to find a certain point (like Winthi HB), you would need to consult an external website like Retorte 
### 4326 is the EPSG code for WGS 84, be carefule: on Retorte, the order is flipped (lat, lon)
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf


osm_id         shop                          name  distance_meters  \
0   706203439  supermarket                          Coop       159.883419   
1  4109460421  supermarket                     Asia Shop       162.391281   
2  3831772784  supermarket                        Migros       247.578208   
3  7380954145  supermarket                      Alnatura       256.838011   
4  4095400190  supermarket                          ALDI       274.275393   
5  4125136758  supermarket  Tandoor Indischer Supermarkt       290.212664   
6  4095400136  supermarket                        Denner       316.354037   
7   709022324  supermarket               Claro Weltladen       441.129317   
8  4058248551  supermarket                        Migros       600.117307   
9  3441033104  supermarket                L'Ultimo Bacio       680.202961   

                       geom  
0  POINT (8.72594 47.50085)  
1  POINT (8.72208 47.50101)  
2  POINT (8.72115 47.49916)  
3  POINT (8.72074 47.49958)  
4   POINT (8.72476 47.4979)  
5  POINT (8.72017 47.50073)  
6  POINT (8.72036 47.49886)  
7  POINT (8.72912 47.49842)  
8  POINT (8.73193 47.50012)  
9  POINT (8.73299 47.49999)

## Show selected features on map

In [11]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query: Select all roads classified as 'motorway' and create a 5000m buffer around these roads.

In [13]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
### revise this query! the "bubble" comes from the query, not the plotting of the map function
sql = """-- Create buffer around major roads
        SELECT 
            1 as group_id,
            ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 5000)), 4326) AS geom
        FROM public.planet_osm_roads AS p
        WHERE
            highway = 'motorway';"""

### the ST_buffer cast 'p.way' as a geometry shape with a 5000m buffer around it
### the ST_union then merges all the shapes into one shape
### the ST_transform then transforms the shape into the coordinate system type SRID 4326 -> WGS84

# Query the database and store the result in a GeoDataFrame
gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

## Show selected features on map

In [14]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=9, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')